# YOLO v3 Object Detection

Let's see how to use the state of the art in object detection! Please make sure to watch the video, there is no code along here, since we can't reasonably train the YOLOv3 network ourself, instead we will use a pre-established version.

CODE SOURCE: https://github.com/xiaochus/YOLOv3

REFERENCE (for original YOLOv3): 

        @article{YOLOv3,  
              title={YOLOv3: An Incremental Improvement},  
              author={J Redmon, A Farhadi },
              year={2018} 
--------
----------
## YOU MUST WATCH THE VIDEO LECTURE TO PROPERLY SET UP THE MODEL AND WEIGHTS. THIS NOTEBOOK WON'T WORK UNLESS YOU FOLLOW THE EXACT SET UP SHOWN IN THE VIDEO LECTURE.
-------
-------

In [1]:
import os
import time
import cv2
import numpy as np

from model.yolo_model import YOLO

In [2]:
def process_image(img):
    """
    Resize, reduce and expand image.

    Argument:
        img: original image.

    Returns
        image: ndarray(64, 64, 3), processed image.
    """
    image = cv2.resize(img, (416, 416), interpolation=cv2.INTER_CUBIC)
    image = np.array(image, dtype='float32')
    image /= 255.
    image = np.expand_dims(image, axis=0)
    return image

In [3]:
def get_classes(file):
    """
    Get classes name.

    Argument:
        file: classes name for database.

    Returns
        class_names: List, classes name.

    """
    with open(file) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

In [4]:
def draw(image, boxes, scores, classes, all_classes):
    """
    Draw the boxes on the image.

    Argument:
        image: original image.
        boxes: ndarray, boxes of objects.
        classes: ndarray, classes of objects.
        scores: ndarray, scores of objects.
        all_classes: all classes name.
    """
    for box, score, cl in zip(boxes, scores, classes):
        x, y, w, h = box

        top = max(0, np.floor(x + 0.5).astype(int))
        left = max(0, np.floor(y + 0.5).astype(int))
        right = min(image.shape[1], np.floor(x + w + 0.5).astype(int))
        bottom = min(image.shape[0], np.floor(y + h + 0.5).astype(int))

        cv2.rectangle(image, (top, left), (right, bottom), (255, 0, 0), 2)
        cv2.putText(
            img=image,
            text=f'{all_classes[cl]} {score:.2f}',
            org=(top, left - 6),
            fontFace=cv2.FONT_HERSHEY_SIMPLEX,
            fontScale=0.6,
            color=(0, 0, 255),
            thickness=1,
            lineType=cv2.LINE_AA
        )

        print(f'class: {all_classes[cl]}, score: {score:.2f}')
        print(f'box coordinate x,y,w,h: {box}')

    print()

In [5]:
def detect_image(image, yolo, all_classes):
    """
    Use YOLO v3 to detect images.

    Argument:
        image: original image.
        yolo: YOLO, yolo model.
        all_classes: all classes name.

    Returns:
        image: processed image.
    """
    pimage = process_image(image)

    start = time.time()
    boxes, classes, scores = yolo.predict(pimage, image.shape)
    duration = start - time.time()

    print(f'time: {duration:.2f}s')

    if boxes is not None:
        draw(image, boxes, scores, classes, all_classes)

    return image

In [6]:
def detect_video(video, yolo, all_classes):
    """
    Use yolo v3 to detect video.

    Argument:
        video: video file.
        yolo: YOLO, yolo model.
        all_classes: all classes name.
    """
    video_path = os.path.join("videos", "test", video)
    camera = cv2.VideoCapture(video_path)
    cv2.namedWindow("detection", cv2.WINDOW_AUTOSIZE)

    # Prepare for saving the detected video
    sz = (
        int(camera.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))
    )
    fourcc = cv2.VideoWriter_fourcc(*'mpeg')
    
    vout = cv2.VideoWriter()
    vout.open(os.path.join("videos", "res", video), fourcc, 20, sz, True)

    while True:
        res, frame = camera.read()
        if not res:
            break

        image = detect_image(frame, yolo, all_classes)
        cv2.imshow("detection", image)

        # Save the video frame by frame
        vout.write(image)

        if cv2.waitKey(110) & 0xff == 27:
            break

    vout.release()
    camera.release()


In [7]:
yolo = YOLO(0.6, 0.5)
file = 'data/coco_classes.txt'
all_classes = get_classes(file)

### Detecting Images

In [10]:
f = 'Vienna_International_Airport.jpg'
path = 'images/' + f
image = cv2.imread(path)
image = detect_image(image, yolo, all_classes)
cv2.imwrite('images/res/' + f, image)

1/1 [==============================] - 2s 2s/step
time: -2.62s
class: aeroplane, score: 0.88
box coordinate x,y,w,h: [ 503.08788776 1953.30535507 1455.9091053   585.39310408]
class: aeroplane, score: 0.87
box coordinate x,y,w,h: [3464.23764038 2326.76891804 1264.7638073   818.58309031]
class: aeroplane, score: 0.77
box coordinate x,y,w,h: [4334.68429565 1751.11431503 1331.14110565  391.74861097]



True

# Detecting on Video

In [11]:
# # detect videos one at a time in videos/test folder    
# video = 'library1.mp4'
# detect_video(video, yolo, all_classes)